In [3]:
paper_path = "paper_data/"
# import modules
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
import tqdm.notebook as tq
import csv
import pickle # for saving results and loading them
from BESTest.py import DCSBMentropy, BESTest, BESTest_confidence

ModuleNotFoundError: No module named 'BESTest'

In [ ]:
def DCSBMentropy(A, g):
  '''
  Input: 
    - adjacency matrix A of numpy sparse matrix type
    - g is a vector of partitions, integers
  Output:
    - h entropies np array
    - p value
  '''
  K = np.max(g) + 1
  e = np.zeros((K, K), dtype=np.double)
  for i in range(K):
    for j in range(K):
      e[i, j] = np.sum(A[g == i][:, g == j])
  m = np.sum(e, axis=1)
  e_sum = np.sum(m)
  for i in range(K):
    for j in range(K):
      e[i, j] /= m[i] * m[j]
  k = np.sum(A, axis=0)
  p = np.matmul(k.T, k) / e_sum
  p *= e[g][:, g]
  pp = np.triu(p)
  pp = pp[pp > 0]
  b = np.size(pp)
  h = 0.5 * ((b-1) * math.log(2 * math.pi * math.e * e_sum) + np.sum(np.log(pp)))
  return h, p

def BESTest(A, g, N = 10000):
  '''
  Input:
    - adjacency matrix A of numpy sparse matrix type
    - g is a vector of partitions, integers
    - N number of samples
  Output:
    - p-value
  '''
  h = np.zeros(shape = (N, ), dtype=np.double)
  h_partition, _ = DCSBMentropy(A,g);
  for i in range(N):
    permuted = np.copy(g)
    np.random.shuffle(permuted)
    h[i], _ = DCSBMentropy(A, permuted)
  p = np.sum(h <= h_partition) / N
  return p

def BESTest_confidence(A, g, N = 10000, n_conf = 30):
  pvals = np.array([BESTest(A, g, N) for _ in range(n_conf)])
  return np.mean(pvals), np.std(pvals)



In [ ]:
def save_obj(obj, name ):
   with open(name + '.pkl', 'wb') as f:
       pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def read_csv(filename):
  reader = csv.reader(open(filename, "r"), delimiter=",")
  x = list(reader)
  result = np.array(x).astype("int")
  return result

In [ ]:
m_friends = read_csv(paper_path + "friend.csv")
m_work = read_csv(paper_path + "work.csv")
m_advice = read_csv(paper_path + "advice.csv")
office = read_csv(paper_path + "office.csv").reshape(-1,) - 1 #because matlab starts from 1, while python starts from 0
gender = read_csv(paper_path + "gender.csv").reshape(-1,) - 1 #reshape is because it loads as (n, 1) while code is for (n, )
school = read_csv(paper_path + "school.csv").reshape(-1,) - 1
practice = read_csv(paper_path + "practice.csv").reshape(-1,) - 1
status = read_csv(paper_path + "status.csv").reshape(-1,) - 1

In [ ]:
networks_matrices = [m_friends, m_work, m_advice]
attributes = [status, gender, office, practice, school]

In [ ]:
p_values_papers = np.zeros((3,5), dtype=np.float64)
sd_values_papers = np.zeros((3,5), dtype=np.float64)

In [ ]:
for i, network in enumerate(networks_matrices):
  for j, attribute in enumerate(attributes):
    p_values_papers[i, j], sd_values_papers[i, j] = BESTest_confidence(network, attribute)  #if want to fast run add N=10, n_conf=3

In [ ]:
# These results are the same as in the original code when we change the model they are using to the same one
# which we are planning to use.
# See the screenshot and the matlab code I used in Network Science Project/replication_proof
# on Google Drive

# writing was done from their data from MATLAB: writematrix(adj_friend,'friend.csv'), etc.

#+/- is the standard deviation over 30 iterations (please see BESTest_confidence())

print("Network    | Status           | Gender           | Office           | Practice         | Law School       |")
for i in range(3):
  line = ['Friendship |', 'Cowork     |', 'Advice     |'][i]
  for j in range(5):
    line += f'{p_values_papers[i,j]:6.4f}(+/-{sd_values_papers[i,j]:6.4f}) |'
  print(line)